In [1]:
import os

In [2]:
%pwd

'd:\\minor(chicken)\\Chiken-Disease-Classification-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\minor(chicken)\\Chiken-Disease-Classification-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from chickenDisease.constants import *
from chickenDisease.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=str(config.root_dir),
            tensorboard_root_log_dir=str(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=str(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import time

In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [13]:
pip install scipy


  Using cached scipy-1.10.1-cp38-cp38-win_amd64.whl.metadata (58 kB)
   ---------------------------------------- 0.0/42.2 MB ? eta -:--:--
   ---------------------------------------- 0.5/42.2 MB 4.2 MB/s eta 0:00:10
   - -------------------------------------- 1.8/42.2 MB 5.3 MB/s eta 0:00:08
   -- ------------------------------------- 3.1/42.2 MB 5.8 MB/s eta 0:00:07
   ---- ----------------------------------- 4.5/42.2 MB 6.0 MB/s eta 0:00:07
   ---- ----------------------------------- 5.2/42.2 MB 6.1 MB/s eta 0:00:07
   ------ --------------------------------- 6.8/42.2 MB 5.7 MB/s eta 0:00:07
   ------- -------------------------------- 7.9/42.2 MB 5.6 MB/s eta 0:00:07
   -------- ------------------------------- 9.2/42.2 MB 5.7 MB/s eta 0:00:06
   --------- ------------------------------ 10.5/42.2 MB 5.7 MB/s eta 0:00:06
   ----------- ---------------------------- 11.8/42.2 MB 5.8 MB/s eta 0:00:06
   ------------ --------------------------- 13.1/42.2 MB 5.8 MB/s eta 0:00:05
   --------

In [19]:
pip install pillow


  Using cached pillow-10.4.0-cp38-cp38-win_amd64.whl.metadata (9.3 kB)
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.6 MB 5.6 MB/s eta 0:00:01
   ---------------------------- ----------- 1.8/2.6 MB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 5.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [12]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-11-15 02:22:10,320: INFO: common: YAML file 'config\config.yaml' loaded successfully]
[2025-11-15 02:22:10,322: INFO: common: YAML file 'params.yaml' loaded successfully]
[2025-11-15 02:22:10,324: INFO: common: created directory at: artifacts]
[2025-11-15 02:22:10,325: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-11-15 02:22:10,326: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-11-15 02:22:10,329: INFO: common: created directory at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/10
19/19 [==============================] - 28s 1s/step - loss: 15.0522 - accuracy: 0.5135 - val_loss: 0.5636 - val_accuracy: 0.9062
Epoch 2/10


d:\anaconda\envs\chicken\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


19/19 [==============================] - 26s 1s/step - loss: 7.8752 - accuracy: 0.6020 - val_loss: 12.5901 - val_accuracy: 0.6094
Epoch 3/10
19/19 [==============================] - 26s 1s/step - loss: 3.3860 - accuracy: 0.7399 - val_loss: 0.8591 - val_accuracy: 0.9219
Epoch 4/10
19/19 [==============================] - 26s 1s/step - loss: 3.4102 - accuracy: 0.7500 - val_loss: 1.1457 - val_accuracy: 0.8594
Epoch 5/10
19/19 [==============================] - 26s 1s/step - loss: 1.2297 - accuracy: 0.8547 - val_loss: 1.4160 - val_accuracy: 0.7969
Epoch 6/10
19/19 [==============================] - 26s 1s/step - loss: 1.3363 - accuracy: 0.8649 - val_loss: 1.0009 - val_accuracy: 0.8906
Epoch 7/10
19/19 [==============================] - 26s 1s/step - loss: 0.6162 - accuracy: 0.9189 - val_loss: 1.4300 - val_accuracy: 0.7969
Epoch 8/10
19/19 [==============================] - 26s 1s/step - loss: 1.0080 - accuracy: 0.9054 - val_loss: 1.2177 - val_accuracy: 0.8281
Epoch 9/10
19/19 [============